<h4>  This notebook is used to subset finalized Grant data by CPC Code from ZIP fiels</h4>
<h4>  Author: Jeffrey Magouirk</h4>
<h4>  Date: 06-29-2022</h4>

In [1]:
import findspark 
findspark.init('/opt/cloudera/parcels/CDH/lib/spark') 

import sys 
import os
import pandas as pd
from pyspark.sql import functions as f
from pyspark.sql import SparkSession 
from pyspark import SparkContext, SparkConf 
from pyspark.sql import HiveContext
from common_functions import time_log_start,time_log_finish
time_log_start()
job_name = 'Connection Test' 
conf = ( 

    SparkConf() 

        .setAppName(job_name) 

.set("hive.metastore.uris","https://bdr3-itwp-1.fqt.uspto.gov:9083") 
.set("spark.sql.extensions","com.qubole.spark.hiveacid.HiveAcidAutoConvertExtension") 
.set("spark.datasource.hive.warehouse.read.via.llap","false") 
.set("spark.sql.hive.llap","true") 
.set("spark.sql.hive.hwc.execution.mode","spark") 
.set("spark.kryo.registrator","com.qubole.spark.hiveacid.util.HiveAcidKyroRegistrator") 
.set("ssl","True") 
.set("spark.datasource.hive.warehouse.read.jdbc.mode","cluster") 
.set("spark.security.credentials.hiveserver2.enabled","false") 
.set("pyFiles","/opt/cloudera/parcels/CDH/lib/hive_warehouse_connector/pyspark_hwc-1.0.0.7.1.5.0-257.zip") 
.set("spark.jars","/opt/cloudera/parcels/CDH/lib/hive_warehouse_connector/hive-warehouse-connector-assembly-1.0.0.7.1.5.0-257.jar")  
.set("spark.sql.hive.hiveserver2.jdbc.url","jdbc:hive2://bdr3-itwp-2.fqt.uspto.gov:8443/default;ssl=true;transportMode=http;httpPath=gateway/cdp-proxy-api/hive")
.set('spark.hadoop.hive.metastore.sasl.enabled','true') 

    ) 

 

spark = (SparkSession.builder.config(conf=conf).enableHiveSupport().getOrCreate()) 

all_databases_df = spark.sql("show databases") 

all_databases_df.show(n=15) 

time_log_finish() 

Start of date and time of this section of code
2022-06-29 14:08:33
+------------------+
|      databaseName|
+------------------+
|               bdr|
|              csam|
|           default|
|               drp|
|information_schema|
|       patent_data|
|              ptab|
|               sys|
|              tept|
|                tm|
|               tqr|
|               trm|
|    trm_databridge|
|      trm_eogadmin|
|    trm_proceeding|
+------------------+
only showing top 15 rows

Finish of date and time of this section of code
2022-06-29 14:08:51


<h4> Reading in the SCHEMA xml files

In [12]:
file_path1 = '/data/jupyter/Jeff_M/CAML/testing_folder/data_xml_schema'
list_dir = os.listdir(file_path1)
list_dir = [y for y in list_dir if y !='.xml'in y]
print('Number of xml files - ',len(list_dir))
print(list_dir[0:10])

Number of xml files -  684
['cpc-scheme-G16B.xml', 'cpc-scheme-D05C.xml', 'cpc-scheme-D01H.xml', 'cpc-scheme-C30B.xml', 'cpc-scheme-Y02T.xml', 'cpc-scheme-B29C.xml', 'cpc-scheme-G08B.xml', 'cpc-scheme-H05H.xml', 'cpc-scheme-Y02C.xml', 'cpc-scheme-F16K.xml']


In [8]:
os.chdir(file_path)

<h4> Works very well, to find the CPC code and cpc items

In [13]:
time_log_start() 
df = pd.DataFrame()
df_B = pd.DataFrame()
df_all = pd.DataFrame()
for i in range(len(list_dir)):
    fn = os.path.join(file_path1,list_dir[i])
    from bs4 import BeautifulSoup as bs
    content = []
# Read the XML file
    with open(fn, "r") as file:
# Read each line in the file, readlines() returns a list of lines
        content = file.readlines()
    # Combine the lines in the list into a string
        content = "".join(content)
        bs_content = bs(content, "lxml")    
    list_cs = bs_content.find_all('classification-symbol')
    list_ci = bs_content.find_all('classification-item')
    print("File name", list_dir[i])
    print('Count of files finished = ',i)
    print('Number of Classification Symbols =',len(list_cs))
    for cs in list_cs:
        var = cs.text
        dict_1 = {'cpc_code':[var]}
        df_0 = pd.DataFrame(dict_1)
        df = df.append(df_0)
    df.head()
    print('Number of Classification items =',len(list_ci))
    for ci in list_ci:
        var = ci.text
        dict_1 = {'cpc_item':[var]}
        df_0 = pd.DataFrame(dict_1)
        df_B = df_B.append(df_0)
    df_B.head()
    df_00 = pd.concat([df, df_B], axis=1)
    df_all = df_all.append(df_00)
    df_all = df_all.drop_duplicates()
    print('Dataframe shape =',df_all.shape)
    file_path = '/data/jupyter/Jeff_M/CAML/testing_folder/data_xml_schema_out'
    file_out = os.path.join(file_path,'Schema_CPC_codes.csv')
    df_all.to_csv(file_out)
    
time_log_finish() 
   

Start of date and time of this section of code
2022-06-29 14:18:16
File name cpc-scheme-G16B.xml
Count of files finished =  0
Number of Classification Symbols = 39
Number of Classification items = 39
Dataframe shape = (39, 2)
File name cpc-scheme-D05C.xml
Count of files finished =  1
Number of Classification Symbols = 77
Number of Classification items = 77
Dataframe shape = (116, 2)
File name cpc-scheme-D01H.xml
Count of files finished =  2
Number of Classification Symbols = 311
Number of Classification items = 311
Dataframe shape = (427, 2)
File name cpc-scheme-C30B.xml
Count of files finished =  3
Number of Classification Symbols = 233
Number of Classification items = 233
Dataframe shape = (660, 2)
File name cpc-scheme-Y02T.xml
Count of files finished =  4
Number of Classification Symbols = 41
Number of Classification items = 41
Dataframe shape = (701, 2)
File name cpc-scheme-B29C.xml
Count of files finished =  5
Number of Classification Symbols = 4584
Number of Classification items 

<h3> Reading in the Definitaions zip xml file

In [ ]:
import zipfile
file_path = '/data/jupyter/Jeff_M/CAML/testing_folder/Final_Schema_XML'
list_dir = os.listdir(file_path)
print(list_dir[0])




In [40]:
#file_get = os.path.join(file_path,list_dir[0])
#newzip=zipfile.ZipFile(file_get,'r')
#newzip.extractall(#)
list_dir2 = os.listdir(file_path)
print('Total Number of files - ',len(list_dir2))
x1 = [y for y in list_dir if y !='.xml'in y]
print('Total Number of xml files - ',len(x1))


Total Number of files -  637
Total Number of xml files -  635


In [ ]:
spark.stop()

<h4> Read in the XML files definitions

In [4]:
### Reading in the XMLs into BeautifulSoup by creating a UDF

def make_beautiful(xml):
    return bs(xml,'lxml').text

parsed_xml_udf = f.udf(make_beautiful)

In [42]:
os.chdir(file_path)
i = 0
fp = os.path.join(os.getcwd(),x1[i])
fp


'/data/jupyter/Jeff_M/CAML/testing_folder/Final_Schema_XML/cpc-definition-D01D.xml'

In [227]:
os.getcwd()
os.chdir('/data/jupyter/Jeff_M/CAML/testing_folder/data_cpc_codes')
os.listdir()

['cpc_codes_full_all.csv', '.ipynb_checkpoints']

In [22]:
df1 = pd.DataFrame()
df = pd.DataFrame()
file_path = '/data/jupyter/Jeff_M/CAML/testing_folder/data_xml_schema'
file_name = '/data/jupyter/Jeff_M/CAML/testing_folder/data_xml_schema_out/xml_Final_schema_out.csv'
#os.listdir(file_path)

In [ ]:
<h4> Reading in the xml files with the CPC code definitions

In [48]:
import bs4
import lxml
df1=pd.DataFrame()
for k in range(len(x1)):
#for k in range(len(x)):
##112 has been done
    fn = os.path.join(file_path,x1[k])
    from bs4 import BeautifulSoup as bs
    content = []
# Read the XML file
    with open(fn, "r") as file:
    # Read each line in the file, readlines() returns a list of lines
        content = file.readlines()
    #content = content[4]
    # Combine the lines in the list into a string
        content = "".join(content)
        bs_content = bs(content, "lxml")
#bs_content   
    import re
    list_cs =bs_content.find_all('classification-symbol')
    print(len(list_cs))
    n_cs = len(list_cs)
    list_df =bs_content.find_all('definition-title')
    print(len(list_df))
    n_df = len(list_df)
    list_ds = bs_content.find_all('definition-statement')
    print(len(list_ds))
    n_ds=len(list_ds)

    n_2 = n_df - n_cs
    n_final = n_2 - n_ds

    if n_final == 0: 
        for i in range(0,n_ds):
            cpc_code =''.join(str(e) for e in list_cs[i])
            cpc_def = ''.join(str(e) for e in list_df[i])
            cpc_def = cpc_def.replace('<class-ref scheme="cpc">','')
            cpc_def = cpc_def.replace('</class-ref>','')
            cpc_state = ''.join(str(e) for e in list_ds[i])
            cpc_state = cpc_state.replace('<section-title>',"")
            cpc_state = cpc_state.replace('</section-title>',"")
            cpc_state = cpc_state.replace('Definition statement',"")
            cpc_state = cpc_state.replace('<section-body>',"")
            cpc_state = cpc_state.replace('</section-body>',"")
            cpc_state = cpc_state.replace('<paragraph-text type="preamble">',"")
            cpc_state = cpc_state.replace('</paragraph-text type="preamble">',"")
            cpc_state = cpc_state.replace('</paragraph-text>',"")
            cpc_state = cpc_state.replace('<paragraph-text type="body">',"")
            cpc_state = cpc_state.replace("This place covers:","")
            dict = {'cpc_codes':[cpc_code],
                'cpc_defns': [cpc_def],
                'cpc_statement':[cpc_state]}
            df = pd.DataFrame(dict)
            df1 = df.append(df1)
            dict = {'cpc_codes':[cpc_code],
                'cpc_defns': [cpc_def],
                'cpc_statement':[cpc_state]}
            df = pd.DataFrame(dict)
        df1 = df.append(df1)
    
        print(df1.shape)
        df1 = df1.drop_duplicates()
        print('Equal =',df1.shape)  
        df_spark = spark.createDataFrame(df1)
        print(df_spark.count())
        df_spark.show(n=2,truncate=False,vertical=True)
        df_spark.write.mode('overwrite').parquet('cpc_codes.parquet')
        df1.to_csv(file_name, index = None, header=True) ###change to the same name as below
    else:
        for i in range(0,n_ds):
            cpc_code =''.join(str(e) for e in list_cs[i])
            cpc_def = ''.join(str(e) for e in list_df[i])
            cpc_def = cpc_def.replace('<class-ref scheme="cpc">','')
            cpc_def = cpc_def.replace('</class-ref>','')
            cpc_state = ''.join(str(e) for e in list_ds[i])
            cpc_state = cpc_state.replace('<section-title>',"")
            cpc_state = cpc_state.replace('</section-title>',"")
            cpc_state = cpc_state.replace('Definition statement',"")
            cpc_state = cpc_state.replace('<section-body>',"")
            cpc_state = cpc_state.replace('</section-body>',"")
            cpc_state = cpc_state.replace('<paragraph-text type="preamble">',"")
            cpc_state = cpc_state.replace('</paragraph-text type="preamble">',"")
            cpc_state = cpc_state.replace('</paragraph-text>',"")
            cpc_state = cpc_state.replace('<paragraph-text type="body">',"")
            cpc_state = cpc_state.replace("This place covers:","")
            dict = {'cpc_codes':[cpc_code],
                'cpc_defns': [cpc_def],
                'cpc_statement':[cpc_state]}
            df = pd.DataFrame(dict)
            df1 = df.append(df1)
            dict = {'cpc_codes':[cpc_code],
                'cpc_defns': [cpc_def],
                'cpc_statement':[cpc_state]}
            df = pd.DataFrame(dict)
        df1 = df.append(df1)
    
        print('No equal =',df1.shape)
        df1 = df1.drop_duplicates()
        print('No equal =',df1.shape ) 
        for j in range(n_ds,n_cs):
            cpc_code =''.join(str(e) for e in list_cs[j])
            cpc_def = ''.join(str(e) for e in list_df[j])
            cpc_state =' '
            dict = {'cpc_codes':[cpc_code],
                'cpc_defns': [cpc_def],
                'cpc_statement':[cpc_state]}
            df = pd.DataFrame(dict)
            df1 = df.append(df1)
            df1.shape
            df1 = df1.drop_duplicates()
            print('Final shape =',df1.shape)
        #df1.to_csv(r'cpc_codes_full.csv', index = None, header=True) 
    
        df_spark = spark.createDataFrame(df1)
        #print(df_spark.count())
        #df_spark.show(n=2,truncate=False,vertical=True)
        df_spark.write.mode('overwrite').parquet('cpc_codes.parquet')
    print(df1.shape)
    df1.to_csv(file_name, index = None, header=True) ##change to the same name as above    

7
7
7
No equal = (8, 3)
No equal = (7, 3)
(7, 3)
42
42
15
No equal = (23, 3)
No equal = (22, 3)
Final shape = (23, 3)
Final shape = (24, 3)
Final shape = (25, 3)
Final shape = (26, 3)
Final shape = (27, 3)
Final shape = (28, 3)
Final shape = (29, 3)
Final shape = (30, 3)
Final shape = (31, 3)
Final shape = (32, 3)
Final shape = (33, 3)
Final shape = (34, 3)
Final shape = (35, 3)
Final shape = (36, 3)
Final shape = (37, 3)
Final shape = (38, 3)
Final shape = (39, 3)
Final shape = (40, 3)
Final shape = (41, 3)
Final shape = (42, 3)
Final shape = (43, 3)
Final shape = (44, 3)
Final shape = (45, 3)
Final shape = (46, 3)
Final shape = (47, 3)
Final shape = (48, 3)
Final shape = (49, 3)
(49, 3)
14
14
12
No equal = (62, 3)
No equal = (61, 3)
Final shape = (62, 3)
Final shape = (63, 3)
(63, 3)
8
8
7
No equal = (71, 3)
No equal = (70, 3)
Final shape = (71, 3)
(71, 3)
10
10
10
No equal = (82, 3)
No equal = (81, 3)
(81, 3)
309
309
84
No equal = (166, 3)
No equal = (165, 3)
Final shape = (166, 3)


In [181]:
print(df1.shape)
#df1.to_csv(r'cpc_codes_full.csv', index = None, header=True) 

(45423, 3)


In [185]:
df1.tail(n=13)

,cpc_codes,cpc_defns,cpc_statement
0,D07B1/0606,{Reinforcing cords for rubber or plastic artic...,electric conductors or elements for signal and...
0,D07B1/025,"{comprising high modulus, or high tenacity, po...",e.g. elements for indicating or detecting wear...
0,D07B1/02,Ropes built-up from fibrous or filamentary mat...,only artificial filamentary material including...
0,D07B1/005,"{Composite ropes, i.e. ropes built-up from fib...",ropes built-up from fibrous or filamentary mat...
0,D07B1/00,Constructional features of ropes or cables,constructional features of ropes and cables.
0,D07B,ROPES OR CABLES IN GENERAL (joining ropes or ...,Ropes and cables and the associated machines a...
0,D01D13/00,Complete machines for producing artificial thr...,\n\nMachines including both mechanical apparat...
0,D01D10/00,Physical treatment of artificial filaments or ...,\n\nPhysical treatment of the filaments or the...
0,D01D7/00,Collecting the newly-spun products ({<class-r...,\n\nCollecting the filaments or the like. Mech...
0,D01D5/00,"Formation of filaments, threads, or the like",\n\nFormation of filaments or the like\n
